# Wordle Solver

In [13]:
from wordle.utils import Wordle

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Play Wordle

In [16]:
# Initialize Wordle class
wordle = Wordle()

# Main function that returns a list of possible words
green_letters = "rt"
green_positions = [2, 5]
yellow_letters = "trrt"
yellow_positions = [1, 4, 3, 4]
gray_letters = "iaebosgun"

# Find words in a list of possible words
words = wordle.find_words(
    green_letters,
    green_positions,
    yellow_letters,
    yellow_positions,
    gray_letters,
    answer_word_list=True,
)

# Choose a word to play
wordle.choose_word_to_play(word_threshold=2)

Number of possible words: 1
crypt 0.0


In [3]:
# Repetitive letters
letters_df = wordle.repetitive_letters()
print(letters_df.to_string(index=False))

Letters  Count
      S      2
      H      2
      A      2
      Y      2
      D      1
      K      1


In [4]:
df = wordle.load_data()
df

,Names,Games_Won
0,Murilo,320
1,Barbara,135
2,Draw,316


In [5]:
# Update score
score_df = wordle.score()
score_df

,Names,Games_Won
0,Murilo,320
1,Barbara,136
2,Draw,316


In [6]:
# Print results
print(score_df.to_string(index=False))

  Names  Games_Won
 Murilo        320
Barbara        136
   Draw        316


#### Git commit message

In [7]:
# git commit message
commit_message = wordle.get_commit_message()
print(commit_message)

git commit -m "updated wordle 320-136-316"


#### Set custom score

In [8]:
# help(wordle.reset_score)

In [9]:
# help(wordle.set_score)

In [10]:
# # Set custom score
# df = wordle.set_score(m_score=311, b_score=132, draw_score=308)
# df

#### Reset score

In [11]:
## Reset score
# wordle.reset_score()